In [1]:
import os
current_dir = os.getcwd()
project_path = os.path.dirname(current_dir)
import sys
if project_path not in sys.path:
    sys.path.append(project_path)

In [2]:
print(current_dir)

/home/glen/2025-LLM-Project


In [9]:
from tqdm import tqdm
from src.retrieval.retrieval_main import retrieval_main
from src.utils.config import load_config
from src.loader.loader_main import loader_main
from src.embedding.embedding_main import embedding_main
from src.retrieval.retrieval_main import retrieval_main
from src.generator.generator_main import generator_main, generate_with_clarification
from src.embedding.embedding_main import generate_index_name

config_path = os.path.join(current_dir, "config.yaml")
print(config_path)
config_origin = load_config(config_path)
chunks = loader_main(config_origin)

/home/glen/2025-LLM-Project/config.yaml

📄 [Verbose] 최종 설정 내용:
data:
  folder_path: data/files
  data_list_path: data/data_list.csv
  top_k: 100
  file_type: all
  apply_ocr: false
  splitter: section
  chunk_size: 1000
  chunk_overlap: 250
embedding:
  embed_model: nlpai-lab/KoE5
  db_type: faiss
  vector_db_path: data
retriever:
  search_type: similarity
  query: 한국 철도 공사와 관련된 문서가 3개인데, 문서마다 차이점이 뭐지?
  top_k: 5
  rerank: true
  min_chunks: 3
generator:
  model_type: huggingface
  model_name: gpt-4.1-nano
  max_length: 512
  use_quantization: true
settings:
  verbose: true



In [10]:
# import yaml
# with open(config_path, 'r', encoding='utf-8') as f:
#     config = yaml.safe_load(f)
print(config_origin)

{'data': {'folder_path': 'data/files', 'data_list_path': 'data/data_list.csv', 'top_k': 100, 'file_type': 'all', 'apply_ocr': False, 'splitter': 'section', 'chunk_size': 1000, 'chunk_overlap': 250}, 'embedding': {'embed_model': 'nlpai-lab/KoE5', 'db_type': 'faiss', 'vector_db_path': 'data'}, 'retriever': {'search_type': 'similarity', 'query': '한국 철도 공사와 관련된 문서가 3개인데, 문서마다 차이점이 뭐지?', 'top_k': 5, 'rerank': True, 'min_chunks': 3}, 'generator': {'model_type': 'huggingface', 'model_name': 'gpt-4.1-nano', 'max_length': 512, 'use_quantization': True}, 'settings': {'verbose': True}}


In [11]:
vector_store = embedding_main(config_origin, chunks, is_save=False)

✅ Vector DB 로드 완료


In [13]:
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langchain_teddynote.messages import messages_to_history

# State definition
class GraphState(TypedDict):
    question: Annotated[str, "Question"]
    context: Annotated[str, "Context"]
    answer: Annotated[str, "Answer"]
    messages: Annotated[list, add_messages]

def retrieve_document(state: GraphState) -> GraphState:
    
    retrieved_docs = retrieval_main(config_origin, vector_store, chunks)
    # Save in context key
    return {"context": retrieved_docs}


# Answer node
def llm_answer(state: GraphState) -> GraphState:
    # Retrieve question
    latest_question = state["question"]

    # This chain only takes one argument(question)
    response = retrieval_chain.invoke(latest_question)

    # Store the generated answer and question in dictionary
    return {
        "answer": response,
        "messages": [("user", latest_question), ("assistant", response)],
    }

from langgraph.checkpoint.memory import MemorySaver

# LangGraph workflow node initialization
workflow = StateGraph(GraphState)

workflow.add_node("retrieve", retrieve_document)
workflow.add_node("llm_answer", llm_answer)

# Workflow chain
workflow.add_edge("retrieve", "llm_answer")
workflow.add_edge("llm_answer", END)

# Start point
workflow.set_entry_point("retrieve")

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import invoke_graph, stream_graph, random_uuid

# config setting(recursion_limit, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

In [16]:
config

{'recursion_limit': 20,
 'configurable': {'thread_id': 'd5e7c3ee-b280-4a16-a711-c259c7b2653e'}}

In [23]:
def format_docs(docs):
    print(docs)
    return "\n\n".join(doc.page_content for doc in docs)

In [22]:
retriever = vector_store.as_retriever(
                search_type="similarity",
                search_kwargs={"k": 3}
            )

In [29]:
from langchain_core.prompts import PromptTemplate

template = (
            "당신은 정부 및 대학의 공공 사업 제안서를 분석하는 AI 전문가입니다.\n"
            "아래 문서 내용은 특정 사업의 목적, 예산, 수행 방식 등을 요약한 것입니다.\n\n"
            "다음 질문에 대해 다음 원칙에 따라 명확하고 정확하게 답변하세요:\n"
            "- 반드시 문서 내용에 기반해서만 답하세요.\n"
            "- 문서에 정보가 없으면 '해당 문서에 정보가 없습니다.'라고 말하세요.\n"
            "- 불확실하거나 추측되는 내용은 포함하지 마세요.\n"
            "- 답변은 최대 5문장 이내로 작성하세요.\n"
            "- 항목이 여러 개인 경우, 항목별로 줄바꿈하여 나열하세요.\n\n"
            "### 문서 내용:\n{context}\n\n"
            "### 질문:\n{question}\n\n"
            "### 답변:"
        )
prompt = PromptTemplate.from_template(template)

In [35]:
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, pipeline)
from langchain_huggingface import (ChatHuggingFace, HuggingFaceEmbeddings,
                                    HuggingFacePipeline)
import torch

language_model_name = "Bllossom/llama-3.2-Korean-Bllossom-AICA-5B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True,
)

model = AutoModelForCausalLM.from_pretrained(
    language_model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(language_model_name)

llm_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.7,
    repetition_penalty=1.2,
    top_p = 0.95,
    return_full_text=False,
    max_new_tokens=512,
)
llm = HuggingFacePipeline(pipeline=llm_pipeline, model_id=language_model_name)

Loading checkpoint shards: 100%|██████████| 3/3 [00:19<00:00,  6.60s/it]
Device set to use cuda:0


In [36]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
inputs = GraphState(question="한국 철도 공사와 관련된 파일이 3개가 있는데, 파일마다 차이점이 뭐지?")
invoke_graph(app, inputs, config)


📌 기존 문서 순서:
  1. 파일명: 한국철도공사 (용역)_예약발매시스템 개량 ISMP 용역.hwp, 청크: 76
  2. 파일명: 한국철도공사 (용역)_예약발매시스템 개량 ISMP 용역.hwp, 청크: 0
  3. 파일명: 한국철도공사 (용역)_예약발매시스템 개량 ISMP 용역.hwp, 청크: 64
  4. 파일명: 한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.hwp, 청크: 24
  5. 파일명: 한국철도공사 (용역)_모바일오피스 시스템 고도화 용역(총체 및 1차).hwp, 청크: 153
  6. 파일명: 한국철도공사 (용역)_모바일오피스 시스템 고도화 용역(총체 및 1차).hwp, 청크: 154
  7. 파일명: 한국철도공사 (용역)_모바일오피스 시스템 고도화 용역(총체 및 1차).hwp, 청크: 152
  8. 파일명: 한국철도공사 (용역)_모바일오피스 시스템 고도화 용역(총체 및 1차).hwp, 청크: 140
  9. 파일명: 한국철도공사 (용역)_[재공고][긴급][협상형]운행정보기록 자동분석시스.hwp, 청크: 117
  10. 파일명: 한국철도공사 (용역)_[재공고][긴급][협상형]운행정보기록 자동분석시스.hwp, 청크: 118
  11. 파일명: 한국철도공사 (용역)_[재공고][긴급][협상형]운행정보기록 자동분석시스.hwp, 청크: 116
  12. 파일명: 한국철도공사 (용역)_[재공고][긴급][협상형]운행정보기록 자동분석시스.hwp, 청크: 104
  13. 파일명: 한국철도공사 (용역)_[재공고][긴급][협상형]운행정보기록 자동분석시스.hwp, 청크: 1
  14. 파일명: 한국철도공사 (용역)_[재공고][긴급][협상형]운행정보기록 자동분석시스.hwp, 청크: 134
  15. 파일명: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf, 청크: 54
  16. 파일명: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf, 청크: 0
  17. 파일명: 기초과학연구원_2025년도 중이온